In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.signal as ss
import sys
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(module_path)
import warnings
warnings.filterwarnings("ignore")

import tools.data_reader_popane as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing

from scipy.fft import fft, fftfreq, fftshift

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

In [5]:
# FEATURE EXTRACTION WITH HEARTPY - BPM, RMSSD, IBI
# WORKS
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

import heartpy as hp

convert_sr = False
studies = {
    1: dr.Study1,
    3: dr.Study3, 
    4: dr.Study4, 
    5: dr.Study5
}
is_clean_ecg = True
signal = dr.Signals.ECG
# measure = "bpm"
# measure = "rmssd"
# measure = "ibi"
# measure = "sdnn"
measure = "breathingrate"
fs = 1000.0
f_dim = preprocessing.DATA_TYPE_DIMENSIONS["Heart"]

for study_num in studies.keys():
    print(f"STUDY: {study_num}")
    phases = studies[study_num].ALL
    subjects = dr.get_subjects(study_num)
    for phase in phases:
        print(f"PHASE: {phase}")
        df = []
        for subject in subjects:
            # print(f"SUBJECT: {subject}")
            try:
                data = dr.get_data_for_subject(study_num, subject, phase, signal).to_numpy()
                # data = data.flatten()
                working_data, measures = hp.process_segmentwise(data, fs, segment_width=55, segment_overlap=5/55)
                value = measures[f"{measure}"]
                value = np.insert(value, 0, subject)
                df.append(value)
            except Exception as e:
                continue
        
        df = pd.DataFrame(df)
        file = os.path.join(dr.Paths.METRICS, f"Study{study_num}", f"{measure}_{phase}.csv")
        with open(file, "w+") as f:
            df.to_csv(f)
            


STUDY: 1
PHASE: Baseline
PHASE: Positive_Emotion_High_Approach
PHASE: Threat
PHASE: Neutral8
STUDY: 3
PHASE: Baseline
PHASE: Gratitude
PHASE: Neutral9
STUDY: 4
PHASE: Baseline
PHASE: Fear2
PHASE: Neutral10
STUDY: 5
PHASE: Anger1
PHASE: Anger2
PHASE: Anger3
PHASE: Neutral1
PHASE: Neutral6
PHASE: Neutral7
PHASE: Amusement2
PHASE: Amusement3
PHASE: Amusement4


In [13]:
# FEATURE EXTRACTION WITH HEARTPY - LF_RR, HF_RR
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

convert_sr = False
studies = {
    1: dr.Study1,
    # 3: dr.Study3, 
    # 4: dr.Study4, 
    # 5: dr.Study5
}
is_clean_ecg = True
signal = dr.Signals.ECG
fs = 1000.0
f_dim = preprocessing.DATA_TYPE_DIMENSIONS["Heart"]

for study_num in studies.keys():
    print(f"STUDY: {study_num}")
    phases = studies[study_num].ALL
    subjects = dr.get_subjects(study_num)
    for phase in phases:
        print(f"PHASE: {phase}")
        metrics_dict = {
            "lf_rr": [],
            "hf_rr": [],
        }
        for subject in subjects:
            # print(f"SUBJECT: {subject}")
            try:
                data = dr.get_data_for_subject(study_num, subject, phase, signal).to_numpy()
                # data = data.flatten()
                ecg_signal = preprocessing.clean_ecg(data)
                lf_rr = preprocessing.get_lf_rr(ecg_signal, fs, window_size=25)
                lf_rr = np.insert(lf_rr, 0, subject)
                lf_rr = pd.DataFrame(lf_rr).dropna(axis=1)

                hf_rr = preprocessing.get_lf_rr(ecg_signal, fs, window_size=25)
                hf_rr = np.insert(hf_rr, 0, subject)
                hf_rr = pd.DataFrame(hf_rr).dropna(axis=1)
                
                metrics_dict["lf_rr"].append(lf_rr)
                metrics_dict["hf_rr"].append(hf_rr)
            except Exception as e:
                continue
        
        for metric in list(metrics_dict.keys()):
            df = pd.concat(metrics_dict[metric], axis=1)
            df = df.transpose()
            file = os.path.join(dr.Paths.METRICS, f"Study{study_num}", f"{metric}_{phase}.csv")
            with open(file, "w+") as f:
                df.to_csv(f)
            


STUDY: 1
PHASE: Baseline
PHASE: Positive_Emotion_High_Approach
PHASE: Threat
PHASE: Neutral8


In [ ]:
# FEATURE EXTRACTION - EDA
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

import samplerate

convert_sr = False
studies = {
    1: dr.Study1,
    # 3: dr.Study3, 
    # 4: dr.Study4, 
    # 5: dr.Study5
}
signal = dr.Signals.EDA
fs = 250.0

for study_num in studies.keys():
    print(f"STUDY: {study_num}")
    phases = studies[study_num].ALL
    subjects = dr.get_subjects(study_num)
    for phase in phases:
        print(f"PHASE: {phase}")
        metrics_dict = {
            "mean_SCL": [],
            "SCR_rate": [],
        }
        for subject in subjects:
            print(f"SUBJECT: {subject}")
            try:
                data = dr.get_data_for_subject(study_num, subject, phase, signal).to_numpy()
                data = samplerate.resample(data, ratio=250.0/1000.0)
                # data = data.flatten()
                mean_scl = preprocessing.get_mean_SCL(data, fs, window_size=25)
                mean_scl = np.insert(mean_scl, 0, subject)
                mean_scl = pd.DataFrame(mean_scl).dropna(axis=1)

                scr_rate = preprocessing.get_SCR_rate(data, fs, window_size=25)
                scr_rate = np.insert(scr_rate, 0, subject)
                scr_rate = pd.DataFrame(scr_rate).dropna(axis=1)
                metrics_dict["mean_SCL"].append(mean_scl)
                metrics_dict["SCR_rate"].append(scr_rate)
            except Exception as e:
                print(e)
                continue

        for metric in list(metrics_dict.keys()):
            df = pd.concat(metrics_dict[metric], axis=1)
            df = df.transpose()
            print(df.head())
            file = os.path.join(dr.Paths.METRICS, f"Study{study_num}", f"{metric}_{phase}.csv")
            with open(file, "w+") as f:
                df.to_csv(f)
            


In [8]:
# FEATURE EXTRACTION - SELF-REPORTS
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

import samplerate

studies = {
    1: dr.Study1,
    3: dr.Study3, 
    4: dr.Study4, 
    5: dr.Study5
}

signal = "affect"

for study_num in studies.keys():
    print(f"STUDY: {study_num}")
    phases = studies[study_num].ALL
    columns = ["subject"] + phases
    subjects = dr.get_subjects(study_num)
    self_report_df = []
    for subject in subjects:
        subject_reports = [subject]
        for phase in phases:
            try:
                data = dr.get_data_for_subject(study_num, subject, phase, signal).to_numpy()
                data = samplerate.resample(data, ratio=250.0/1000.0)
                self_report = data.mean()
                subject_reports.append(self_report)
            except Exception as e:
                subject_reports.append(-1)
        self_report_df.append(subject_reports)
    self_report_df = pd.DataFrame(self_report_df, columns=columns)
    # print(self_report_df.head())
    file = os.path.join(dr.Paths.METRICS, f"Study{study_num}", f"self_reports.csv")
    with open(file, "w+") as f:
        self_report_df.to_csv(f)

STUDY: 1
STUDY: 3
STUDY: 4
STUDY: 5
